In [51]:
import json
from typing import Sequence, List, Dict
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
import openai
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import nest_asyncio
from dotenv import load_dotenv

In [52]:
from chatcompletion import llm
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [53]:
system_message = """You are a text analyst responsible for analyzing the summary of a research article and responsible for finding relationship between keywords.
    The main aim is to derive new key ideas/discoveries from the provided research reviews for key related terms.
    Analyse the full text very carefully since some relationships can be found from differnt article and in that case they should have one  main topic.
    To plot the graph using a table format, you can structure the data in a way that lists each connection between a main key words and its subtopics. 
    This table should have two columns: one for the "Main Topic" and another for the "Subtopic". Each row will represent an edge (connection) in the graph.
    Please make sure the entry names are short but very clear and understandable and also make sure the nodes are very important for research discoveries and analysis.
    Please make sure the final JSON can be read by pandas and must be enclosed with [ ].
    Please make sure the final output is JSON with column names, "Main Topic" and "Subtopic".
    """

with open("summaries.txt", "r") as file:
    infomat = file.read()

In [54]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f""" {system_message}"""},
    {"role": "user", "content": f""" {infomat}"""}
  ]
)

In [55]:
response = completion.choices[0].message.content

print(response)

```json
[
  {"Main Topic": "Brain Metastasis", "Subtopic": "Overall Survival"},
  {"Main Topic": "Brain Metastasis", "Subtopic": "Local Control"},
  {"Main Topic": "Brain Metastasis", "Subtopic": "Distant Brain Control"},
  {"Main Topic": "Radiotherapy", "Subtopic": "WBRT vs Stereotactic"},
  {"Main Topic": "Radiotherapy", "Subtopic": "Postoperative Targeted Radiotherapy"},
  {"Main Topic": "Left-sided Breast Cancer", "Subtopic": "DIBH Benefits"},
  {"Main Topic": "DIBH", "Subtopic": "Heart and Lung Dose Reduction"},
  {"Main Topic": "DIBH", "Subtopic": "Cardiac Substructures"},
  {"Main Topic": "Dosimetry", "Subtopic": "MLC Adjustment"},
  {"Main Topic": "PDIP-AAA", "Subtopic": "Treatment Plan Accuracy"},
  {"Main Topic": "PDIP-AAA", "Subtopic": "Gamma Passing Rates"},
  {"Main Topic": "NSCLC", "Subtopic": "Atezolizumab and Varlilumab"},
  {"Main Topic": "NSCLC", "Subtopic": "Palliative Radiation"},
  {"Main Topic": "TASK-3 Channel", "Subtopic": "Anesthetics"},
  {"Main Topic": "TASK-

In [56]:

cleaned_output = response.replace("```json", "").replace("```", "").strip()
file_path = 'cleaned_output.json'

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(cleaned_output)


In [57]:
import pandas as pd
df = pd.read_json("cleaned_output.json")

In [58]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import itertools

# Assuming df is already defined and contains the data

G = nx.DiGraph()

# Build the graph
for index, row in df.iterrows():
    G.add_node(row['Main Topic'], type='Main Topic')
    G.add_node(row['Subtopic'], type='Subtopic')
    G.add_edge(row['Main Topic'], row['Subtopic'])

# Generate positions for nodes
pos = nx.spring_layout(G, iterations=2000, k=0.07)

colors = itertools.cycle(mcolors.CSS4_COLORS)

node_colors = {}

# Assign a unique color to each connected component
for component in nx.weakly_connected_components(G):
    color = next(colors)
    for node in component:
        node_colors[node] = color

# Create node traces for Plotly, only including Main Topic nodes
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        size=[], 
        color=[],
        opacity=0.9,
        line=dict(width=3, color='rgba(0, 0, 0, 0.5)'),
        symbol='circle',
    ))

# Add the main topic nodes and their positions to the node_trace
for node in G.nodes():
    if G.nodes[node]['type'] == 'Main Topic':
        x, y = pos[node]
        node_trace['x'] += (x,)
        node_trace['y'] += (y,)
        
        # Gather subtopics and format them with each subtopic on a new line using <br> for line breaks
        subtopics = [f"{i+1}. {n}" for i, n in enumerate(G.neighbors(node))]
        hover_text = f"{node}<br>Subtopics:<br>" + "<br>".join(subtopics)
        node_trace['text'] += (hover_text,)
        
        node_size = 25  # Fixed size for Main Topic
        node_trace['marker']['size'] += (node_size,)
        
        node_color = mcolors.to_rgba(node_colors[node], alpha=0.9)  # Semi-transparent color
        node_trace['marker']['color'] += (f"rgba({node_color[0]*255}, {node_color[1]*255}, {node_color[2]*255}, 0.9)",)

# No edge traces or annotations are added

# Create and show the figure
fig = go.Figure(data=[node_trace],
                layout=go.Layout(
                    title='Retina-Style Network Graph',
                    titlefont_size=20,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, visible=False), 
                    yaxis=dict(showgrid=False, zeroline=False, visible=False),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    width=700,
                    height=700
                ))

fig.show()



In [59]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import itertools

# Assuming df is already defined and contains the data

G = nx.DiGraph()

# Build the graph
for index, row in df.iterrows():
    G.add_node(row['Main Topic'], type='Main Topic')
    G.add_node(row['Subtopic'], type='Subtopic')
    G.add_edge(row['Main Topic'], row['Subtopic'])

# Generate positions for nodes with adjusted parameters
pos = nx.spring_layout(G, iterations=100, k=1)

colors = itertools.cycle(mcolors.CSS4_COLORS)

node_colors = {}

# Assign a unique color to each connected component
for component in nx.weakly_connected_components(G):
    color = next(colors)
    for node in component:
        node_colors[node] = color

# Create edge traces for Plotly
edge_trace = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace.append(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=2, color='rgba(150, 150, 150, 0.6)'),
        hoverinfo='none',
        mode='lines'))

# Create node traces for Plotly
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        size=[], 
        color=[],
        opacity=0.9,
        line=dict(width=3, color='rgba(0, 0, 0, 0.5)'),
        symbol='circle',
    ))

# Add the nodes and their positions to the node_trace
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)
    
    # For main topics, set the hover text with subtopics as before
    if G.nodes[node]['type'] == 'Main Topic':
        subtopics = [f"{i+1}. {n}" for i, n in enumerate(G.neighbors(node))]
        hover_text = f"{node}<br>Subtopics:<br>" + "<br>".join(subtopics)
        node_trace['text'] += (hover_text,)
    else:
        node_trace['text'] += ('',)  # Empty hover text for subtopics
    
    node_size = 25 if G.nodes[node]['type'] == 'Main Topic' else 15
    node_trace['marker']['size'] += (node_size,)
    
    node_color = mcolors.to_rgba(node_colors[node], alpha=0.9)  # Semi-transparent color
    node_trace['marker']['color'] += (f"rgba({node_color[0]*255}, {node_color[1]*255}, {node_color[2]*255}, 0.9)",)

annotations = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    annotations.append(dict(
        ax=x0, ay=y0, axref='x', ayref='y',
        x=x1, y=y1, xref='x', yref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=2.5,
        arrowwidth=2,
        arrowcolor='rgba(150, 150, 150, 0.6)'
    ))

# Create and show the figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(
                    title='Retina-Style Network Graph',
                    titlefont_size=20,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=annotations,
                    xaxis=dict(showgrid=False, zeroline=False, visible=False), 
                    yaxis=dict(showgrid=False, zeroline=False, visible=False),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    width=1000,
                    height=1000
                ))

fig.show()


In [60]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import itertools

G = nx.DiGraph()

for index, row in df.iterrows():
    G.add_node(row['Main Topic'], type='Main Topic')
    G.add_node(row['Subtopic'], type='Subtopic')
    G.add_edge(row['Main Topic'], row['Subtopic'])

pos = nx.spring_layout(G, iterations=100, k=1)

colors = itertools.cycle(mcolors.CSS4_COLORS)
node_colors = {}

for component in nx.weakly_connected_components(G):
    color = next(colors)
    for node in component:
        node_colors[node] = color

# Create node traces for Plotly
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        size=[], 
        color=[], 
        opacity=0.9,
        line=dict(width=3, color='rgba(0, 0, 0, 0.5)'),
        symbol='circle',
    ))

# Add the nodes and their positions to the node_trace
for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)
    
    if G.nodes[node]['type'] == 'Main Topic':
        subtopics = [f"{i+1}. {n}" for i, n in enumerate(G.neighbors(node))]
        hover_text = f"{node}<br>Subtopics:<br>" + "<br>".join(subtopics)
        node_trace['text'] += (hover_text,)
    else:
        node_trace['text'] += ('',)
    
    node_size = 30 if G.nodes[node]['type'] == 'Main Topic' else 20
    node_trace['marker']['size'] += (node_size,)
    
    node_color = mcolors.to_rgba(node_colors[node], alpha=0.9)
    node_trace['marker']['color'] += (f"rgba({int(node_color[0]*255)}, {int(node_color[1]*255)}, {int(node_color[2]*255)}, 0.9)",)

# Pre-create edge traces, initially invisible, and use the color of the main topic
edge_traces = []
for edge in G.edges():
    main_topic = edge[0]  # The main topic is the source node of the edge
    color = mcolors.to_rgba(node_colors[main_topic], alpha=0.9)
    
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_traces.append(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=2, color=f'rgba({int(color[0]*255)}, {int(color[1]*255)}, {int(color[2]*255)}, {color[3]})'),
        hoverinfo='none',
        mode='lines',
        visible=False
    ))

# Create buttons for dropdown menu
buttons = []
main_topics = sorted([node for node in G.nodes() if G.nodes[node]['type'] == 'Main Topic'])

for node in main_topics:
    node_edges = [i for i, edge in enumerate(G.edges()) if edge[0] == node]
    button = dict(
        label=node,
        method='update',
        args=[{
            'visible': [False] * len(edge_traces) + [True],
        }]
    )
    for i in node_edges:
        button['args'][0]['visible'][i] = True  # Make the relevant edges visible
        edge_traces[i]['line']['width'] = 6  # Make the edge bold
    buttons.append(button)

# Create and display the figure
fig = go.Figure(data=edge_traces + [node_trace],
                layout=go.Layout(
                    title='Interactive Network Graph',
                    titlefont_size=20,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, visible=False), 
                    yaxis=dict(showgrid=False, zeroline=False, visible=False),
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    width=800,
                    height=800,
                    updatemenus=[{
                        'type': 'dropdown',
                        'showactive': True,
                        'buttons': buttons,
                    }]
                ))

fig.show()
